<a href="https://colab.research.google.com/github/nitish9801/colab/blob/main/TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
tf.__version__

'2.18.0'

In [ ]:
import pandas as pd
pd.__version__

'2.2.2'